In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [4]:
import os
if not os.path.isdir('saved_models_{}'.format('dcgan')):
    os.mkdir('saved_models_{}'.format('dcgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('dcgan'), mode='w')
import cv2
import torch.utils.data as Data

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (X_test, y_test) = fashion_mnist.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        X_test = np.expand_dims(X_test, axis=3)
        # y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # labels = y_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (wants discriminator to mistake images as real)
                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (
                epoch, global_step, d_loss[0], 100 * d_loss[1], g_loss))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % save_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=40, batch_size=64, save_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 64)          0         
__________

epoch:0 step:56 [D loss: 0.822589, acc.: 59.38%] [G loss: 1.455574]
epoch:0 step:57 [D loss: 0.686524, acc.: 60.16%] [G loss: 1.465844]
epoch:0 step:58 [D loss: 0.790365, acc.: 57.81%] [G loss: 1.865590]
epoch:0 step:59 [D loss: 0.727056, acc.: 58.59%] [G loss: 1.680666]
epoch:0 step:60 [D loss: 0.668991, acc.: 65.62%] [G loss: 1.679888]
epoch:0 step:61 [D loss: 0.495998, acc.: 77.34%] [G loss: 1.597771]
epoch:0 step:62 [D loss: 0.523173, acc.: 76.56%] [G loss: 1.553009]
epoch:0 step:63 [D loss: 0.486684, acc.: 76.56%] [G loss: 1.273860]
epoch:0 step:64 [D loss: 0.418755, acc.: 82.03%] [G loss: 0.918257]
epoch:0 step:65 [D loss: 0.491727, acc.: 77.34%] [G loss: 0.978193]
epoch:0 step:66 [D loss: 0.413580, acc.: 83.59%] [G loss: 0.968596]
epoch:0 step:67 [D loss: 0.533194, acc.: 72.66%] [G loss: 0.794507]
epoch:0 step:68 [D loss: 0.821108, acc.: 57.81%] [G loss: 1.206363]
epoch:0 step:69 [D loss: 0.931978, acc.: 52.34%] [G loss: 1.400116]
epoch:0 step:70 [D loss: 0.755480, acc.: 55.47%]

epoch:0 step:179 [D loss: 0.669058, acc.: 65.62%] [G loss: 1.486704]
epoch:0 step:180 [D loss: 0.644264, acc.: 67.97%] [G loss: 1.508768]
epoch:0 step:181 [D loss: 0.728246, acc.: 59.38%] [G loss: 1.317614]
epoch:0 step:182 [D loss: 0.491400, acc.: 80.47%] [G loss: 1.409816]
epoch:0 step:183 [D loss: 0.594720, acc.: 70.31%] [G loss: 1.129401]
epoch:0 step:184 [D loss: 0.528518, acc.: 71.88%] [G loss: 1.474337]
epoch:0 step:185 [D loss: 0.650714, acc.: 60.16%] [G loss: 1.353460]
epoch:0 step:186 [D loss: 0.673439, acc.: 60.94%] [G loss: 1.305401]
epoch:0 step:187 [D loss: 0.534003, acc.: 73.44%] [G loss: 1.560924]
epoch:0 step:188 [D loss: 0.635142, acc.: 67.19%] [G loss: 1.525559]
epoch:0 step:189 [D loss: 0.599141, acc.: 72.66%] [G loss: 1.361362]
epoch:0 step:190 [D loss: 0.497914, acc.: 77.34%] [G loss: 1.597524]
epoch:0 step:191 [D loss: 0.582320, acc.: 67.97%] [G loss: 1.761942]
epoch:0 step:192 [D loss: 0.642452, acc.: 61.72%] [G loss: 1.387023]
epoch:0 step:193 [D loss: 0.673547

epoch:0 step:294 [D loss: 0.535270, acc.: 76.56%] [G loss: 1.508343]
epoch:0 step:295 [D loss: 0.641596, acc.: 62.50%] [G loss: 1.615929]
epoch:0 step:296 [D loss: 0.624925, acc.: 60.94%] [G loss: 1.641127]
epoch:0 step:297 [D loss: 0.807712, acc.: 57.03%] [G loss: 1.451208]
epoch:0 step:298 [D loss: 0.584637, acc.: 67.19%] [G loss: 1.619772]
epoch:0 step:299 [D loss: 0.572973, acc.: 72.66%] [G loss: 1.373768]
epoch:0 step:300 [D loss: 0.653287, acc.: 67.19%] [G loss: 1.563024]
epoch:0 step:301 [D loss: 0.660906, acc.: 63.28%] [G loss: 1.573438]
epoch:0 step:302 [D loss: 0.634319, acc.: 64.84%] [G loss: 1.511027]
epoch:0 step:303 [D loss: 0.518172, acc.: 76.56%] [G loss: 1.878928]
epoch:0 step:304 [D loss: 0.521594, acc.: 73.44%] [G loss: 1.541513]
epoch:0 step:305 [D loss: 0.568094, acc.: 73.44%] [G loss: 1.567520]
epoch:0 step:306 [D loss: 0.613756, acc.: 73.44%] [G loss: 1.465927]
epoch:0 step:307 [D loss: 0.493211, acc.: 76.56%] [G loss: 1.589814]
epoch:0 step:308 [D loss: 0.666883

epoch:0 step:415 [D loss: 0.613664, acc.: 64.84%] [G loss: 1.544177]
epoch:0 step:416 [D loss: 0.616228, acc.: 67.19%] [G loss: 1.663039]
epoch:0 step:417 [D loss: 0.658633, acc.: 68.75%] [G loss: 2.102263]
epoch:0 step:418 [D loss: 0.723777, acc.: 59.38%] [G loss: 2.008166]
epoch:0 step:419 [D loss: 0.436602, acc.: 77.34%] [G loss: 2.113604]
epoch:0 step:420 [D loss: 0.631610, acc.: 62.50%] [G loss: 1.481114]
epoch:0 step:421 [D loss: 0.647165, acc.: 64.06%] [G loss: 1.650931]
epoch:0 step:422 [D loss: 0.487219, acc.: 74.22%] [G loss: 1.782959]
epoch:0 step:423 [D loss: 0.605120, acc.: 66.41%] [G loss: 1.642191]
epoch:0 step:424 [D loss: 0.606437, acc.: 67.19%] [G loss: 1.762268]
epoch:0 step:425 [D loss: 0.592624, acc.: 70.31%] [G loss: 1.964741]
epoch:0 step:426 [D loss: 0.568159, acc.: 75.78%] [G loss: 1.825477]
epoch:0 step:427 [D loss: 0.531305, acc.: 72.66%] [G loss: 1.817023]
epoch:0 step:428 [D loss: 0.557036, acc.: 71.88%] [G loss: 1.573107]
epoch:0 step:429 [D loss: 0.699704

...

epoch:39 step:37015 [D loss: 0.640383, acc.: 70.31%] [G loss: 1.561657]
epoch:39 step:37016 [D loss: 0.395142, acc.: 85.94%] [G loss: 1.897826]
epoch:39 step:37017 [D loss: 0.616814, acc.: 58.59%] [G loss: 1.673984]
epoch:39 step:37018 [D loss: 0.371229, acc.: 89.84%] [G loss: 1.415376]
epoch:39 step:37019 [D loss: 0.407767, acc.: 84.38%] [G loss: 1.546313]
epoch:39 step:37020 [D loss: 0.765074, acc.: 53.12%] [G loss: 1.407418]
epoch:39 step:37021 [D loss: 0.479014, acc.: 78.12%] [G loss: 2.183782]
epoch:39 step:37022 [D loss: 0.400261, acc.: 85.16%] [G loss: 1.390155]
epoch:39 step:37023 [D loss: 0.460024, acc.: 80.47%] [G loss: 1.650183]
epoch:39 step:37024 [D loss: 0.599158, acc.: 67.97%] [G loss: 1.571798]
epoch:39 step:37025 [D loss: 0.461354, acc.: 78.91%] [G loss: 0.969702]
epoch:39 step:37026 [D loss: 0.630423, acc.: 59.38%] [G loss: 1.187783]
epoch:39 step:37027 [D loss: 0.461679, acc.: 77.34%] [G loss: 1.431484]
epoch:39 step:37028 [D loss: 0.333607, acc.: 87.50%] [G loss: 2.

epoch:39 step:37131 [D loss: 0.833844, acc.: 50.00%] [G loss: 1.735230]
epoch:39 step:37132 [D loss: 0.424228, acc.: 79.69%] [G loss: 1.735153]
epoch:39 step:37133 [D loss: 0.527990, acc.: 69.53%] [G loss: 1.305272]
epoch:39 step:37134 [D loss: 0.594800, acc.: 66.41%] [G loss: 1.460369]
epoch:39 step:37135 [D loss: 0.476764, acc.: 78.91%] [G loss: 1.556097]
epoch:39 step:37136 [D loss: 0.665349, acc.: 64.06%] [G loss: 1.770960]
epoch:39 step:37137 [D loss: 0.468082, acc.: 78.91%] [G loss: 1.458035]
epoch:39 step:37138 [D loss: 0.405588, acc.: 86.72%] [G loss: 1.310797]
epoch:39 step:37139 [D loss: 0.477354, acc.: 75.78%] [G loss: 1.664857]
epoch:39 step:37140 [D loss: 0.587402, acc.: 70.31%] [G loss: 2.251184]
epoch:39 step:37141 [D loss: 0.649052, acc.: 57.81%] [G loss: 1.461180]
epoch:39 step:37142 [D loss: 0.416469, acc.: 81.25%] [G loss: 1.467730]
epoch:39 step:37143 [D loss: 0.615809, acc.: 70.31%] [G loss: 1.337894]
epoch:39 step:37144 [D loss: 0.398314, acc.: 84.38%] [G loss: 1.

epoch:39 step:37245 [D loss: 0.295822, acc.: 86.72%] [G loss: 1.932544]
epoch:39 step:37246 [D loss: 0.592523, acc.: 71.88%] [G loss: 1.558492]
epoch:39 step:37247 [D loss: 0.646823, acc.: 63.28%] [G loss: 1.471736]
epoch:39 step:37248 [D loss: 0.378458, acc.: 82.81%] [G loss: 1.464034]
epoch:39 step:37249 [D loss: 0.411010, acc.: 80.47%] [G loss: 1.562377]
epoch:39 step:37250 [D loss: 0.391942, acc.: 83.59%] [G loss: 1.645561]
epoch:39 step:37251 [D loss: 0.555283, acc.: 67.97%] [G loss: 2.166753]
epoch:39 step:37252 [D loss: 0.670113, acc.: 60.94%] [G loss: 1.808284]
epoch:39 step:37253 [D loss: 0.601612, acc.: 67.97%] [G loss: 1.328098]
epoch:39 step:37254 [D loss: 0.420784, acc.: 82.03%] [G loss: 1.497415]
epoch:39 step:37255 [D loss: 0.743246, acc.: 55.47%] [G loss: 1.544561]
epoch:39 step:37256 [D loss: 0.611006, acc.: 67.19%] [G loss: 1.961744]
epoch:39 step:37257 [D loss: 0.655334, acc.: 63.28%] [G loss: 1.932378]
epoch:39 step:37258 [D loss: 0.629850, acc.: 67.97%] [G loss: 1.

epoch:39 step:37360 [D loss: 0.565242, acc.: 68.75%] [G loss: 1.883466]
epoch:39 step:37361 [D loss: 0.468737, acc.: 77.34%] [G loss: 1.365231]
epoch:39 step:37362 [D loss: 0.449409, acc.: 82.03%] [G loss: 1.032136]
epoch:39 step:37363 [D loss: 0.661996, acc.: 61.72%] [G loss: 1.411644]
epoch:39 step:37364 [D loss: 0.603538, acc.: 64.06%] [G loss: 1.124521]
epoch:39 step:37365 [D loss: 0.621184, acc.: 67.19%] [G loss: 1.267666]
epoch:39 step:37366 [D loss: 0.439787, acc.: 78.91%] [G loss: 2.184372]
epoch:39 step:37367 [D loss: 0.472971, acc.: 78.91%] [G loss: 1.758645]
epoch:39 step:37368 [D loss: 0.423298, acc.: 83.59%] [G loss: 1.597846]
epoch:39 step:37369 [D loss: 0.477967, acc.: 76.56%] [G loss: 2.008663]
epoch:39 step:37370 [D loss: 0.449578, acc.: 78.12%] [G loss: 1.768026]
epoch:39 step:37371 [D loss: 0.797803, acc.: 53.91%] [G loss: 1.815733]
epoch:39 step:37372 [D loss: 0.759980, acc.: 54.69%] [G loss: 1.228346]
epoch:39 step:37373 [D loss: 0.317719, acc.: 92.19%] [G loss: 1.

epoch:39 step:37475 [D loss: 0.338060, acc.: 88.28%] [G loss: 1.732601]
epoch:39 step:37476 [D loss: 0.551471, acc.: 71.09%] [G loss: 1.335128]
epoch:39 step:37477 [D loss: 0.562218, acc.: 75.78%] [G loss: 1.649644]
epoch:39 step:37478 [D loss: 0.425856, acc.: 80.47%] [G loss: 1.436218]
epoch:39 step:37479 [D loss: 0.480611, acc.: 76.56%] [G loss: 1.705756]
epoch:39 step:37480 [D loss: 0.584994, acc.: 73.44%] [G loss: 1.306676]
